In [1]:
'''General Libraries'''
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', 500)

'''Statistic'''
import scipy
from scipy import stats
from scipy.stats import norm
import datetime

'''Scikit Learn'''
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV

'''Ploting Libraries'''
from plotly.offline import iplot, plot
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.renderers.default = "notebook" 


import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set_palette('RdBu')

'''Miscellaneous'''
from yellowbrick.regressor import ResidualsPlot
import missingno as msno

'''Seeds'''
import random
random.seed(10)
np.random.seed(11)


In [2]:
df = pd.read_csv('cleanData_to_kPrototypes.csv')

In [3]:
df.head()

,5k,10k,20k,half,25k,30k,35k,40k,official,pace,5k_avgSpeed,10k_avgSpeed,20k_avgSpeed,25k_avgSpeed,30k_avgSpeed,35k_avgSpeed,40k_avgSpeed,rank_overall,rank_in_gender_only,rank_in_division,age,age_division,gender,country,state
0,0.003409,0.007832,0.008234,0.008508,0.007689,0.008533,0.010421,0.010017,0.010147,0.010863,0.003409,0.007832,0.008234,0.007689,0.008533,0.010421,0.010017,0.000219,0.000398,0.001003,0.460317,0.3,M,JPN,others
1,0.106944,0.166667,0.157928,0.158276,0.150604,0.134029,0.128172,0.129555,0.127324,0.126930,0.106944,0.166667,0.157928,0.150604,0.134029,0.128172,0.129555,0.000626,0.000000,0.000000,0.238095,0.0,F,KEN,others
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.365079,0.2,M,RSA,others
3,0.106692,0.166562,0.157928,0.158276,0.150604,0.134029,0.128172,0.131521,0.130881,0.130932,0.106692,0.166562,0.157928,0.150604,0.134029,0.128172,0.131521,0.000814,0.000114,0.000287,0.095238,0.0,F,ETH,others
4,0.003409,0.005221,0.002550,0.002870,0.002344,0.001813,0.001074,0.001434,0.001375,0.001144,0.003409,0.005221,0.002550,0.002344,0.001813,0.001074,0.001434,0.000031,0.000057,0.000143,0.349206,0.2,M,JPN,others


In [4]:
df[['age_division', 'gender', 'country', 'state']] = df[['age_division', 'gender', 'country', 'state']].astype('category')

In [5]:
df.dtypes

5k                      float64
10k                     float64
20k                     float64
half                    float64
25k                     float64
30k                     float64
35k                     float64
40k                     float64
official                float64
pace                    float64
5k_avgSpeed             float64
10k_avgSpeed            float64
20k_avgSpeed            float64
25k_avgSpeed            float64
30k_avgSpeed            float64
35k_avgSpeed            float64
40k_avgSpeed            float64
rank_overall            float64
rank_in_gender_only     float64
rank_in_division        float64
age                     float64
age_division           category
gender                 category
country                category
state                  category
dtype: object

# K-Prototypes Clustering

In [6]:
categories = ['age_division', 'gender', 'country', 'state']

categories = [df.columns.get_loc(col) for col in categories if col in df]

print(categories)

[21, 22, 23, 24]


In [7]:
from kmodes.kmodes import KModes
from kmodes.kprototypes import KPrototypes

n_clusters=3

kp = KPrototypes(n_clusters=n_clusters, init='Huang', n_init=5, verbose=1,
                 n_jobs=-1,
                 max_iter=50,
                 gamma=0.1)
clusters = kp.fit_predict(df, categorical=categories)

# Print the cluster centroids
centroids = kp.cluster_centroids_


Best run was number 4


In [11]:
df['cluster'] = clusters

In [13]:
df.cluster

0        2
1        2
2        2
3        2
4        2
        ..
31643    2
31644    0
31645    0
31646    0
31647    1
Name: cluster, Length: 31648, dtype: uint16

In [14]:
df.to_csv('results_kprototypes_k_3.csv', index=False)